<a href="https://colab.research.google.com/github/Uz8351/MyPhython/blob/master/FLORES_GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

from google.colab import drive
drive.mount('/content/drive')


import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image

# Ruta al directorio de imágenes
data_dir = '/content/drive/MyDrive/druida/flores'

# Dimensiones de las imágenes de entrada que espera InceptionV3
img_size = (299, 299)

# Preprocesamiento de imágenes
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generador de datos para entrenamiento
train_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='training',
    color_mode='rgb',  # Añadir esta línea
    classes=['flores1', 'flores2']
)

# Generador de datos para validación
validation_generator = data_gen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    color_mode='rgb',  # Añadir esta línea
    classes=['flores1', 'flores2']
)


# Cargar el modelo preentrenado InceptionV3 sin las capas densas (fully connected)
base_model = InceptionV3(input_shape=(299, 299, 3), include_top=False)

# Congelar las capas base para que no se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

# Agregar capas personalizadas para la clasificación
x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output_layer = layers.Dense(train_generator.num_classes, activation='softmax')(x)

# Crear el modelo final
model = Model(base_model.input, output_layer)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Ejemplo de clasificación de una imagen
def predict_flower_type(image_path):
    img = load_and_preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)

    if class_idx == 0:
        return 'flores1'
    elif class_idx == 1:
        return 'flores2'
    else:
        return 'Clasificación desconocida'

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    # Implementa la carga y preprocesamiento de la imagen según tus necesidades
    # Aquí, se utiliza el preprocesamiento básico con rescale
    img = image.load_img(image_path, target_size=img_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalizar
    return img,  # <-- Asegurar que la salida sea una tupla

# Ejemplo de clasificación de una imagen
image_filename = 'image_00001.jpg'  # Cambia al nombre de la imagen que deseas clasificar
image_path_to_classify = os.path.join(data_dir, 'flores1', 'image_' + image_filename.split('_')[-1])
predicted_class = predict_flower_type(image_path_to_classify)
print(f"La flor en la imagen pertenece a la clase: {predicted_class}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 37 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 23s 8s/step - loss: 19.2085 - accuracy: 0.5405 - val_loss: 32.4006 - val_accuracy: 0.2500
Epoch 2/10
2/2 [==============================] - 16s 13s/step - loss: 36.7549 - accuracy: 0.5946 - val_loss: 39.7574 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 16s 6s/step - loss: 38.7275 - accuracy: 0.6757 - val_loss: 52.6103 - val_accuracy: 0.6250
Epoch 4/10
2/2 [==============================] - 16s 13s/step - loss: 18.6196 - accuracy: 0.7027 - val_loss: 127.9352 - val_accuracy: 0.2500
Epoch 5/10
2/2 [==============================] - 16s 13s/step - loss: 31.6393 - accuracy: 0.6757 - val_loss: 128.1413 - val_accuracy: 0.2500
Epoch 6/10
2/2 [==============================] - 16s 13s/step - loss: 26.2654 - accura

ValueError: ignored